In [1]:
import pandas as pd

In [2]:
df_raw = pd.read_csv("./toxic.csv")
df_raw.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
TOXIC_LABELS = df_raw.columns.tolist()[2:]
toxic_items = df_raw[df_raw[TOXIC_LABELS].sum(axis=1) > 0]
ok_items = df_raw[df_raw[TOXIC_LABELS].sum(axis=1) == 0]

In [4]:
sample_size = 10000

In [5]:
df = pd.concat([
    toxic_items.sample(sample_size),
    ok_items.sample(sample_size)
])

In [6]:
df["label"] = (df[TOXIC_LABELS].sum(axis=1) > 0).astype(int)
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,label
136698,db4301138dd1df71,Sjö is pathetical. Consist (presently at 83.25...,1,0,0,0,0,0,1
112745,5b3255a04a3c3eff,go to hell you gestapo!,1,0,0,0,0,0,1
72099,c102961adbad28a1,You are such a fucking hypocrite. \n\nI hope y...,1,0,1,1,1,0,1
51188,88f4ed433a57bae3,I guess if the Marvel Directory stuff is copie...,1,0,0,0,0,0,1
75389,c9b0eba0f9d55fbc,P.S Im a fucking cunt rag and enjoy the taste ...,1,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
91014,f377021fcfcbce40,"IIRC, It was Slipknot's self titled that final...",0,0,0,0,0,0,0
98976,1175b771fbe8535c,"""\nThe orange flash is visible to any one look...",0,0,0,0,0,0,0
37523,6439bbb051fda5b3,Final Fantasy series\nThank you for the commen...,0,0,0,0,0,0,0
79716,d5480e0c62c74875,"""::Would you please re-read what you just wrot...",0,0,0,0,0,0,0


In [7]:
from datasets import Dataset

toxic_dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2)
toxic_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'label', '__index_level_0__'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'label', '__index_level_0__'],
        num_rows: 4000
    })
})

In [8]:
from transformers import DistilBertTokenizer

bert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(data):
    return bert_tokenizer(data["comment_text"], truncation=True)

In [9]:
toxic_dataset_tokenized = toxic_dataset.map(tokenize)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [10]:
pd.DataFrame(toxic_dataset_tokenized["test"]).head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,label,__index_level_0__,input_ids,attention_mask
0,a48371da5a3e0ecf,Okay really nigga why does everything have to ...,0,0,1,0,1,1,1,126482,"[101, 3100, 2428, 9152, 23033, 2339, 2515, 267...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,3d7a225900848076,"Meanie ferret \n\nnever revert me again, you p...",1,0,0,0,1,0,1,23267,"[101, 2812, 2666, 10768, 27032, 2196, 7065, 87...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,ed0405a126fc3927,you suck messing with honduras! \n\nFuck you\n...,1,0,1,0,1,0,1,139951,"[101, 2017, 11891, 22308, 2007, 14373, 999, 66...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,5e6bc8683b3e29ed,eat Dl2000CK \n\n..|..,1,0,0,0,0,0,1,149594,"[101, 4521, 21469, 28332, 2692, 3600, 1012, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,3e6281309210d3e0,"""\n Next thing you fuckers will say that those...",1,0,1,0,1,0,1,23621,"[101, 1000, 2279, 2518, 2017, 6616, 2545, 2097...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [11]:
from transformers import DataCollatorWithPadding

collator = DataCollatorWithPadding(bert_tokenizer)

In [12]:
import evaluate

metric = evaluate.load("accuracy")
def calc_metrics(evaluation):
    logits, labels = evaluation
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer
import numpy as np

In [14]:
batch_size = 64
epochs = 3

In [15]:
train_args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    output_dir="./tox/models",
    logging_strategy="epoch",
    logging_dir="./tox/log",
)

In [16]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

bert_classification_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
bert_classification_model.config.id2label = {0: "Ok", 1: "Toxic"}
bert_classification_model.config.label2id = {"Ok": 0, "Toxic": 1}

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
trainer = Trainer(
    model=bert_classification_model,
    processing_class=bert_tokenizer,
    args=train_args,
    train_dataset=toxic_dataset_tokenized["train"],
    eval_dataset=toxic_dataset_tokenized["test"],
    compute_metrics=calc_metrics,
    data_collator=collator,
)

In [18]:
trainer.evaluate()

{'eval_loss': 0.7101767659187317,
 'eval_model_preparation_time': 0.0008,
 'eval_accuracy': 0.50325,
 'eval_runtime': 10.2686,
 'eval_samples_per_second': 389.539,
 'eval_steps_per_second': 6.135}

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,0.203400,0.200012,0.000800,0.924750
2,0.092700,0.171805,0.000800,0.938000
3,0.039000,0.240190,0.000800,0.937750


TrainOutput(global_step=750, training_loss=0.11172829437255859, metrics={'train_runtime': 396.75, 'train_samples_per_second': 120.983, 'train_steps_per_second': 1.89, 'total_flos': 5996153358588672.0, 'train_loss': 0.11172829437255859, 'epoch': 3.0})

In [20]:
trainer.save_model()

In [21]:
api_key = 'hf_foobarbaz'
path = 'foo/bar'

In [23]:
trainer.model.push_to_hub(
    repo_id=path, token=api_key
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ZhengjunHUO/distilbert-toxicity-classifier/commit/de80fe69795a46e0c1e5f6ddd450a26988464331', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='de80fe69795a46e0c1e5f6ddd450a26988464331', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ZhengjunHUO/distilbert-toxicity-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='ZhengjunHUO/distilbert-toxicity-classifier'), pr_revision=None, pr_num=None)

In [25]:
from transformers import pipeline

t_classifier = pipeline(
    'text-classification',
    model=trainer.model,
    tokenizer=bert_tokenizer,
    use_fast=True,
    top_k=None
)

Device set to use cuda:0


In [26]:
t_classifier('whoever wrote this is a waste of space')

[[{'label': 'Toxic', 'score': 0.7278553247451782},
  {'label': 'Ok', 'score': 0.27214473485946655}]]